# AI Multimodal Data Repository

First, we connect to the data respository `aimm.lbl.gov`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tiled.client import from_uri

In [ ]:
client = from_uri("https://aimm.lbl.gov")

Some data is public to all and visible without logging in.

In [ ]:
# print publically available datasets

for k in client:
    print(k)

In [ ]:
client["newville"]

In [ ]:
node = client["newville"]["Fe-K-0"]
node

In [ ]:
data = node.read()
data

In [ ]:
from pprint import pprint

pprint(dict(node.metadata))

In [ ]:
print("Edge:", node.metadata['common']['element']['edge'])
print("Element:", node.metadata['common']['element']['symbol'])

In [ ]:
data.plot("energy", "mutrans")

In [ ]:
# register queries

from tiled.query_registration import QueryTranslationRegistry, register
from dataclasses import dataclass
import collections
import json

@register(name="raw_mongo", overwrite=True)
@dataclass
class RawMongo:
    """
    Run a MongoDB query against a given collection.
    """

    query: str  # We cannot put a dict in a URL, so this a JSON str.

    def __init__(self, query):
        if isinstance(query, collections.abc.Mapping):
            query = json.dumps(query)
        self.query = query
        
@register(name="element", overwrite=True)
@dataclass
class ElementQuery:

    symbol: str
    edge: str

    def __init__(self, symbol, edge):
        self.symbol = symbol
        self.edge = edge

In [ ]:
# use ElementQuery to search for a particular edge
results = client["newville"].search(ElementQuery(symbol="Co", edge="K"))
print(f"{len(results)} results")

In [ ]:
# plot our results

df = results.values_indexer[0].read()
df["mutrans"] = np.log(df["i0"] / df["itrans"])
df.plot("energy", "mutrans")

login to get access to non-public data

In [ ]:
client.context.authenticate();

In [ ]:
# print available datasets

for k in client:
    print(k)

In [ ]:
# look at the heald dataset

heald = client["heald"]
heald

In [ ]:
# search for Cobalt K edges (note we can use the same query)
results = heald.search(ElementQuery(symbol="Co", edge="K"))
print(f"{len(results)} results")

In [ ]:
# look at the first result

df = results.values_indexer[0].read()
df["mutrans"] = np.log(df["i0"] / df["it"])

df.plot("energy", "mutrans")

In [ ]:
# look at core soft XAS dataset from Wanli

wanli = client["wanli"]
wanli

In [ ]:
# look at all Oxygen K edges

results = wanli.search(ElementQuery(symbol="O", edge="K"))
print(f"{len(results)} results")
results

In [ ]:
# print all results
for k in results:
  print(k)

In [ ]:
# compare some results

fig, ax = plt.subplots()

for k in ["FeO", "CoO", "NiO"]:
    df = results[k].read()
    energy = df.energy.to_numpy()
    mu = df.mu.to_numpy() / df.mu.max()
    ax.plot(energy, mu, label=k)
  
ax.legend()

In [ ]:
# look at all Cobalt L edges

results = wanli.search(ElementQuery(symbol="Co", edge="L"))
print(f"{len(results)} results")
results

In [ ]:
# print all results

for k in results:
    print(k)

In [ ]:
# compare some results

fig, ax = plt.subplots()

for k in ["f5195teyn_LiCoO2", "f6163teyn_FullyCharged LiCoO2"]:
    df = results[k].read()
    df.plot("energy", "mu", ax=ax, label=k)

ax.legend()